In [ ]:
WORKING_DIR = "/home/xavier/Documents/DAE_project"

# Get selected frames of selected strains

In [29]:
import os
import cv2
import numpy as np
import pandas as pd

IMG_DIR = f"{WORKING_DIR}/dataset/Roy_training/images"
OUT_DIR = f"{WORKING_DIR}/images/figure1"
REFERENCE_CSV = f"{WORKING_DIR}/dataset/Roy_training/Caro 3d 9.7.22_2.20_new.xlsx"

# Specify strains and frames of interest
strains = [1622, 8615, 2232, 4398, 4299, 5208, 4408]
frames = [1, 361, 721, 1081, 1441]  # Example: specify your target frame numbers here


def resize_crop(img_path, resize_by=1.0, resolution=512, brightness_norm=True, brightness_mean=107):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise FileNotFoundError(f"Image not found: {img_path}")

    if img.dtype != np.uint8:
        img = (img / 256).astype(np.uint8)

    original_shape = img.shape
    resized_shape = (int(original_shape[1] * resize_by), int(original_shape[0] * resize_by))

    if resize_by != 1.0:
        img = cv2.resize(img, resized_shape, interpolation=cv2.INTER_LANCZOS4)

    # Crop the center
    center_crop = img[
        (resized_shape[1] - resolution) // 2:(resized_shape[1] + resolution) // 2,
        (resized_shape[0] - resolution) // 2:(resized_shape[0] + resolution) // 2
    ]

    if brightness_norm:
        current_mean = np.mean(center_crop)
        delta = brightness_mean - current_mean
        center_crop = np.clip(cv2.add(center_crop, delta), 0, 255).astype(np.uint8)

    return center_crop


# Load reference data
reference_df = pd.read_excel(REFERENCE_CSV)

# Build image folder DataFrame
folder_data = []
for folder_name in os.listdir(IMG_DIR):
    if folder_name[-4:].isdigit():
        folder_data.append([int(folder_name[-4:]), folder_name])
folder_df = pd.DataFrame(folder_data, columns=["Run", "Folder"])

# Merge with reference information
image_df = folder_df.merge(reference_df, on="Run", how="left")
image_df["Mutant #"] = image_df["Mutant #"].astype(str)

# Process each target strain
for strain in strains:
    strain = str(strain)
    subset = image_df[image_df["Mutant #"] == strain]
    if subset.empty:
        print(f"Strain {strain} not found in metadata. Skipping.")
        continue

    output_path = os.path.join(OUT_DIR, str(strain))
    os.makedirs(output_path, exist_ok=True)

    for _, experiment in subset.iterrows():
        folder_path = os.path.join(IMG_DIR, experiment["Folder"])
        if not os.path.isdir(folder_path):
            print(f"Folder not found: {folder_path}. Skipping.")
            continue

        for scope in os.listdir(folder_path):
            scope_path = os.path.join(folder_path, scope)
            if not os.path.isdir(scope_path):
                continue
            print(scope_path)
            all_filenames = os.listdir(scope_path)
            frame_dict = {
                int(fname[-8:-4]): fname for fname in all_filenames if fname[-8:-4].isdigit()
            }

            for frame in frames:
                search_frame = frame
                cnt = 1
                while search_frame not in frame_dict:
                    search_frame = frame + (-1) ** cnt * cnt
                    cnt += 1
                    if cnt > 50:
                        print(f"Could not find frame close to {frame} in {scope_path}")
                        break

                if search_frame in frame_dict:
                    filename = frame_dict[search_frame]
                    img_path = os.path.join(scope_path, filename)
                    try:
                        img = resize_crop(img_path)
                        save_path = os.path.join(output_path, f"{strain}_{scope}_{search_frame}.jpg")
                        cv2.imwrite(save_path, img)
                        print(f"Saved {save_path}")
                    except Exception as e:
                        print(f"Failed to process {img_path}: {e}")


/home/xavier/Documents/dataset/Welch/trainingset2/trainingset2/CS5_04_1622_1%agar_Run0582/Scope09
Saved /home/xavier/PycharmProjects/imgs_for_publication/figure1/1622/1622_Scope09_1.jpg
Saved /home/xavier/PycharmProjects/imgs_for_publication/figure1/1622/1622_Scope09_361.jpg
Saved /home/xavier/PycharmProjects/imgs_for_publication/figure1/1622/1622_Scope09_721.jpg
Saved /home/xavier/PycharmProjects/imgs_for_publication/figure1/1622/1622_Scope09_1081.jpg
Saved /home/xavier/PycharmProjects/imgs_for_publication/figure1/1622/1622_Scope09_1441.jpg
/home/xavier/Documents/dataset/Welch/trainingset2/trainingset2/CS2_44_1622_1%agar_Run0320/Scope23
Saved /home/xavier/PycharmProjects/imgs_for_publication/figure1/1622/1622_Scope23_1.jpg
Saved /home/xavier/PycharmProjects/imgs_for_publication/figure1/1622/1622_Scope23_361.jpg
Saved /home/xavier/PycharmProjects/imgs_for_publication/figure1/1622/1622_Scope23_721.jpg
Saved /home/xavier/PycharmProjects/imgs_for_publication/figure1/1622/1622_Scope23_1081